<a href="https://colab.research.google.com/github/psygrammer/psypy/blob/master/notebooks/03_b_funcions_and_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 03. 내장 자료구조, 함수, 파일 (2)

* 싸이그래머 / PsyPy [1]
* 김무성

In [1]:
!python -V

Python 3.6.8


# 차례 
* 함수
    - 네임스페이스, 스코프, 지역 함수
    - 여러 값 반환하기
    - 함수도 객체다
    - 익명 함수
    - 클로저 : 함수를 반환하는 함수
    - \*args와  \*\*kwargs를 사용해서 호출 문법 확장하기  
    - 커링 : 일부 인자만 취하기
    - 제너레이터
    - 에러와 예외 다루기
* 파일과 운영체제
    - 바이트와 유니코드 파일

------------------------

# 함수

```python
def my_function(x, y, z=1.5) :
    if z > 1 :
        return z * (x + y)
    else :
        return z / (x + y)
    
# return문이 몇 개가 되든 상관없다. 
# 함수 블록이 끝날 때까지 return문이 없다면 None이 반환된다.   
```

## 네임스페이스, 스코프, 지역 함수

```python
# 로컬
def func() :
    a = []  # <- local
    for i in range(5) :
        a.append(i)
```        

```python
# 전역 
a = []
def func() :
    for i in range(5) :
        a.append(i)
        
func()
a
```

In [2]:
b = None
def bind_b_variable() :
    b = []
    
bind_b_variable()    
print(b)    

None


In [3]:
# global 예약어 사용
b = None
def bind_b_variable() :
    global b
    b = []

bind_b_variable()
print(b)

[]


```python
# 함수 안에서 함수를 선언할 수 있다.
def outer_function(x, y, z) :
    def inner_function(a, b, c) :
        pass
    pass
```

## 여러 값 반환하기

In [4]:
def f() :
    a = 5
    b = 6
    c = 7
    return a, b, c

a, b, c = f()
print(a, b, c)

5 6 7


In [5]:
return_value = f()
return_value # 여래 개의 값을 하나의 변수에 반환받으면, 그 변수는 튜플 자료형이 된다.

(5, 6, 7)

## 함수도 객체다

In [0]:
states = [ '  Alabama ', 'Georgia!', 'Georgia', 'georgia', 'FlOrIda', 
          'south carolin##', 'West virginaia?']

### 문자열을 변형하기 위해 함수를 사용하는 일반적인 패턴

In [0]:
import re 

def clean_strings(strings) :
    result = []
    for value in strings :
        value = value.strip()
        value = re.sub('[!#?]', '', value) # 문장 부호 제거
        value = value.title()
        result.append(value)
    return result

In [8]:
clean_strings(states)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

### 함수를 객체화하고 인자로 넘겨서 좀 더 유연하게 사용하는 패턴

In [0]:
def remove_punctuation(value) :
    return re.sub('[!#?]', '', value)

# 함수도 객체. 
clean_ops = [str.strip, remove_punctuation, str.title]

def clean_strings(strings, ops) :
    result = []
    for value in strings :
        for function in ops :
            value = function(value) 
        result.append(value)
    return result

In [10]:
clean_strings(states, clean_ops)

['Alabama',
 'Georgia',
 'Georgia',
 'Georgia',
 'Florida',
 'South Carolin',
 'West Virginaia']

In [11]:
# map을 이용해본다. 어떤 컬렉션에 대해 함수를 적용하는 함수다. 
# 이를 이용해서 함수를 인자로 사용할 수 있다.
map(remove_punctuation, states)

In [12]:
for e in map(remove_punctuation, states) :
    print(e)

  Alabama 
Georgia
Georgia
georgia
FlOrIda
south carolin
West virginaia


## 익명 함수 

```python
def short_function(x) :
    return x * 2

# 위의 함수와 동치인 익명 함수(람다 함수)
equiv_anon = lambda x: x * 2
```

In [13]:
# 각 문자열에서 다양한 문자가 포함된 순서로 정렬하는 예제
strings = ['foo', 'card', 'bar', 'aaaa', 'abab']
strings.sort(key=lambda x: len(set(list(x))))
strings

['aaaa', 'foo', 'abab', 'bar', 'card']